## Libraries

In [50]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
import sys

## Data for the collections

In [38]:
literature = pd.read_csv("Literature.csv", sep=';')
biology = pd.read_csv("Biology.csv", sep=';')
philosophy = pd.read_csv("Philosophy.csv", sep=';')

In [39]:
#df to dict
biol = biology.to_dict(orient = "records")
phil = philosophy.to_dict(orient = "records")
lit = literature.to_dict(orient = "records")

##  Connecting and creating the database and collections

In [40]:
client = MongoClient()
db = client.QuoteAPI

In [41]:
biology =  db.Biology
philosophy =  db.Philosophy
literature =  db.Literature

In [42]:
db.Biology.insert_many(biol)
db.Philosophy.insert_many(phil)
db.Literature.insert_many(lit)    

In [94]:
#Para combinar todas las collecciones en una sola, donde solo tengamos la informacion de las frases y los autores
y = db.Biology.aggregate( [
   { '$unionWith': "Philosophy"},
   { '$unionWith': "Literature"},   
   { '$project': { '_id': 1, "Quote":1, "Author":1, "Category":1, "Gender":1}},
   { '$out' : "Quote_Author" } ])

In [44]:
list(db.Quote_Author.find({'Author': 'Aristotle' }))

[{'_id': ObjectId('6033b97d14687f7bb0232fa8'),
  'Author': 'Aristotle',
  'Gender': 'M',
  'Quote': 'By �life,� we mean a thing that can nourish itself and grow and decay',
  'Category': 'science'}]

In [64]:
a = db.Quote_Author.aggregate([
  { "$group": {
    "_id": "$Author",
    "Quotess": {
      "$push": {
        "Quote": "$$ROOT"  #nos permite mantener todos los documentos dl
      }
    }
  }}, { '$out' : "Quotessss" }
])

In [75]:
a = db.Quote_Author.aggregate([{ '$group' : {
            '_id' : "$Author",
            "quotes": { '$push' : "$$ROOT"} }
        }, { '$out' : "Quotessss33" }])

In [95]:
y = db.Biology.aggregate( [
   { '$unionWith': "Philosophy"},
   { '$unionWith': "Literature"},   
   { "$group": {
    "_id": "$Author",
    "Quotes": {
      "$push": {
        "Quote": "$$ROOT"  #nos permite mantener todos los documentos dl
      }
    }
  }}, 
   { '$out' : "All_Quotes" } ])

In [87]:
list(db.All_Quotes.find({'_id': ' Socrates'}))

[{'_id': ' Socrates',
  'Quotes': [{'Quote': {'_id': ObjectId('6033b97d14687f7bb0232fbd'),
     'Quote': 'The unexamined life is not worth living ',
     'Author': ' Socrates',
     'Gender': 'M',
     'Category': 'philosophy'}},
   {'Quote': {'_id': ObjectId('6033b97d14687f7bb0232fd0'),
     'Quote': 'There is only one good, knowledge, and one evil, ignorance ',
     'Author': ' Socrates',
     'Gender': 'M',
     'Category': 'philosophy'}},
   {'Quote': {'_id': ObjectId('6033b97d14687f7bb0232fef'),
     'Quote': 'The only thing I know is that I know nothing ',
     'Author': ' Socrates',
     'Gender': 'M',
     'Category': 'philosophy'}}]}]